In [1]:
import cv2,os,time
os.sys.path.append('../utilities/')
from datetime import datetime
import numpy as np
import IPython.display as Disp
import support_utility_openvino
import matplotlib.pyplot as plt
import ipywidgets as widgets


In [2]:
output_support = support_utility_openvino.create_plot()

In [3]:
person_model_path = '/home/prateek/shared_space/useful_models/openvino/person-detection-0202/FP32/person-detection-0202.xml'
# person_model_path = "/home/prateek/prateek_space/intel_model_dump/intel/person-detection-retail-0002/FP32/person-detection-retail-0002.xml"
ppe_model_path = "/media/prateek/prateek_space/model_files/UTCL_ppe/gas_chamber/models/ppe_no_ppe/openvino_inceptionV3/frozen_model.xml"
ppe_model_path = "/media/prateek/shared_space/useful_models/utcl_model/person_crop_model/resnet50_FPN_ssd/openvino_model/frozen_inference_graph.xml"
threshold = .5
device = 'CPU'

person_detection = support_utility_openvino.async_infer(4)
person_detection.load_model(model_path=person_model_path,device=device)

ppe_detection = support_utility_openvino.async_infer(4,ie_network=person_detection.ie)
ppe_detection.load_model(model_path=ppe_model_path,device=device)



Available Devices :  ['CPU', 'GNA', 'GPU']
OPTIMIZATION_CAPABILITIES for CPU:  ['FP32', 'FP16', 'INT8', 'BIN']
model inputs : dict_keys(['image'])
model outputs :  dict_keys(['detection_out'])
Available Devices :  ['CPU', 'GNA', 'GPU']
OPTIMIZATION_CAPABILITIES for CPU:  ['FP32', 'FP16', 'INT8', 'BIN']
model inputs : dict_keys(['image_tensor'])
model outputs :  dict_keys(['DetectionOutput'])


In [4]:
ppe_class = ['helmet', 'hood', 'balacava', 'no_helmet', 'visor', 'jacket', 'vest', 'no_vest', 'worker', 'HeavyJacket']
res_range = None

In [5]:
channel = "/home/prateek/shared_space/useful_vids/UTCL/VID-20210120-WA0076.mp4"
cap = cv2.VideoCapture(channel)
ret,frame = cap.read()

cv2.namedWindow("preview",cv2.WINDOW_NORMAL)
ret,frame = cap.read()
fps = []
thresh = .2
if res_range is None:
    res_range = output_support.get_resolution_thresh(frame)
person_detection.reinit_model()
ppe_detection.reinit_model()
write_video = False
offset = 10
if write_video:
    fourcc = cv2.VideoWriter_fourcc(*'h264')
    
    videowriter = cv2.VideoWriter("../output_vids/UTCL/PPE/"+"gas_chamber"+".mp4",fourcc, 20, (int(cap.get(3)),int(cap.get(4))))

while True:
    fps = fps
#     Disp.clear_output(wait=True)
    t1 = time.time()
    ret,frame = cap.read()
    if not ret:
        break
    
    person_detection.predict(frame)
    if person_detection.frame_processed>=person_detection.num_requests:
        try:
            frame,attr,res = person_detection.postprocess_op()
#             res[0] = np.expand_dims(res[0][res[0][:,:,:,1]==2],[0,1])
            op,bboxes,res_filt = output_support.trim_frame_with_result(frame,res[0],threshold=thresh,return_results = True,resolution_thresh_range = res_range,offset=offset)
            attrs = ppe_detection.predict_batch_async(op)
            attrs_text = [ppe_class[np.argmax(att[0])] for att in attrs]
#             dets.append(len(attrs))
            fps.append(1/(time.time()-t1))
            op_frame = output_support.write_text(frame,"FPS = "+str(np.mean(fps,dtype='int')),text_color = (0,0,0),font_scale=1,font_thickness=2,highlight_color = (127,0,255))
            for idx,b in enumerate(bboxes):
                box_color = (0,255,0)
                text = attrs_text[idx]
                if "no_" in text:
                    box_color = (0,0,255)
                op_frame = output_support.create_bbox_with_text(op_frame,(tuple(b[:2]),tuple(b[2:])),text,box_color=box_color)
#             op_frame = output_support.label_obj_detection(frame,res[0],labels=['vehicle','person','car'],font_scale=1,font_thickness = 1,thresh=.5)
        except Exception as e:
            print(e)
#         print(person_detection.cursor_id,person_detection.frame_processed)
        cv2.imshow("preview",op_frame)
        k = cv2.waitKey(1)
        if k==ord('q'):
            break
        if write_video:
            videowriter.write(op_frame)
# cap.release()
if write_video:
    videowriter.release()
cap.release()
cv2.destroyAllWindows()
        
        
    

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range


In [35]:
res_range

(0.013937114197530864, 0.044376929012345676)